In [1]:
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
import re
import numpy as np

In [2]:
url = 'https://www.gov.za/speeches/statement-cabinet-meeting-wednesday-24-november-2021-25-nov-2021-0000'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'lxml')

In [3]:
languages = ['af','nr','xh','zu','st','nso','tn','ss','ve','ts last']

In [4]:
total_news = []

for i in range(11):
    url_page = 'https://www.gov.za/cabinet-statements?page=' + str(i)
    req_page = Request(url_page,headers=hdr)
    page_page = urlopen(req_page)
    soup_page = BeautifulSoup(page_page, 'lxml')
    
    for table_url in soup_page.findAll('td', {'class':'views-field views-field-title-field'}):
        slug = table_url.find('a')['href']
        print(slug)
        
        url = 'https://www.gov.za' + slug
        req = Request(url,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page, 'lxml')

        title = soup.findAll('h1',{'class':'title'})[0].text
        date = soup.findAll('span',{'class':'date-display-single'})[0]
        datetime_str = date['content']
        date_str = date.text
        text = soup.findAll('div',{'class':'field field-name-body field-type-text-with-summary field-label-hidden'})[0].text.replace('\xa0',' ')
        if len(soup.findAll('li',{'class':languages[0]})) ==1:
            news_item = {}
            news_item['title'] = title
            news_item['url'] = url
            news_item['date'] = date_str
            news_item['datetime'] = datetime_str
            news_item['en'] = {'text':text,'title':title, 'url':url}
            for lang in languages:
                url_trans = 'https://www.gov.za' + soup.findAll('li',{'class':lang})[0].findAll('a',{'class':'language-link'})[0]['href']
                req_trans = Request(url_trans,headers=hdr)
                page_trans = urlopen(req_trans)
                soup_trans = BeautifulSoup(page_trans, 'lxml')
                title_trans = soup_trans.findAll('h1',{'class':'title'})[0].text
                text_trans = soup_trans.findAll('div',{'class':'field field-name-body field-type-text-with-summary field-label-hidden'})[0].text.replace('\xa0',' ')
                news_item[lang] = {'text':text_trans, 'title':title_trans, 'url':url_trans}
            total_news.append(news_item)
            print("Saved: ", title)

/speeches/statement-virtual-cabinet-meeting-21-september-2022-23-sep-2022-0000
Saved:  Sitatemende seMhlangano weKhabhinethi ngevidiyo ye-inthanethi  ngaLesitsatfu, mhla tinge-21 INyoni 2022
/speeches/statement-cabinet-discussions-energy-21-september-2022-21-sep-2022-0000
/speeches/statement-virtual-cabinet-meeting-31-august-2022-1-sep-2022-0000
Saved:  Statement on the virtual Cabinet Meeting of 31 August 2022
/speeches/statement-virtual-cabinet-meeting-wednesday-%C2%A017-august-2022-13-aug-2022-0000
Saved:  Statement on the virtual Cabinet Meeting of 17 August 2022
/speeches/statement-virtual-cabinet-meeting-held-thursday-7-july-2022-11-jul-2022-0000
Saved:  Statement on the virtual Cabinet Meeting of 7 July 2022
/speeches/statement-virtual-cabinet-meeting-wednesday-22-june-2022-23-jun-2022-0000
Saved:  Statement on the Cabinet Meeting of 22 June 2022
/speeches/statement-cabinet-meeting-08-june-2022-9-jun-2022-0000
Saved:  Statement on the Cabinet Meeting of 8 June 2022
/speeches/sta

In [5]:
len(total_news)

159

In [6]:
import json

In [7]:
final = json.dumps(total_news, indent=2)

In [8]:
with open("govza-cabinet-statements.json", "w") as final:
   json.dump(total_news, final)

In [9]:
speeches_data = pd.read_json('govza-cabinet-statements.json')

In [10]:
speeches_data

,title,url,date,datetime,en,af,nr,xh,zu,st,nso,tn,ss,ve,ts last
0,Sitatemende seMhlangano weKhabhinethi ngevidiy...,https://www.gov.za/speeches/statement-virtual-...,2022-09-23,2022-09-23 00:00:00+02:00,{'text': 'A. Tindzaba Letimcoka Telive1. Teman...,{'text': 'A. Kwessies in die omgewing1. Energi...,{'text': 'A. Ezingundabamlonyeni 1. Zama...,{'text': 'A. Imiba Engundabamlonyeni1. A...,{'text': 'A. Ezisematheni1. Ezamandla1.1...,{'text': 'A. Maemo a Renang1. Motlakase1...,{'text': 'A. Ditaba tše di Amago Badudi1. Mohl...,{'text': 'A. Merero ya ga Jaanong1. Motlakase1...,{'text': 'A. Tindzaba Letimcoka Telive1. Teman...,{'text': 'A. Mafhungo a zwino kha Shango 1....,{'text': 'A. Xiyimo xa Khabinete eka Timhak...
1,Statement on the virtual Cabinet Meeting of 31...,https://www.gov.za/speeches/statement-virtual-...,2022-09-01,2022-09-01 00:00:00+02:00,{'text': 'A. Issues in the environment1. Eco...,{'text': 'A. Kwessies in die Omgewing1. ...,{'text': 'A. Ezingundabamlonyeni 1. Zomn...,{'text': 'A. Imiba enguNdaba-mlonyeni1. ...,{'text': 'A. Ezisematheni1. Umnotho 1.1....,{'text': 'A. Maemo a Renang ha Jwale1. Moruo1....,{'text': 'A. Ditaba tše di Amago Badudi1. Ekon...,{'text': 'A. Merero ya ga Jaanong1. Ikonomi1.1...,{'text': 'A. Tindzaba Letibalulekile telive...,{'text': '1. Ikonomi1.1. Khabinethe yo ṱangane...,{'text': 'A. Xiyimo xa Khabinete eka Timhak...
2,Statement on the virtual Cabinet Meeting of 17...,https://www.gov.za/speeches/statement-virtual-...,2022-08-13,2022-08-13 00:00:00+02:00,{'text': 'Issues in the Environment1. Energy 1...,{'text': 'A. Kwessies in die omgewing 1. Krag ...,{'text': 'A. Ezingundabamlonyeni1. Zegezi 1.1....,{'text': 'Issues in the Environment1. Energy 1...,{'text': 'A. Ezisematheni1. Ezamandla 1.1. IKh...,{'text': 'A. Maemo a Renang ha Jwale1. Motla...,{'text': 'A. Ditaba tšeo di Amago Badudi1. ...,{'text': 'A. Merero ya ga Jaanong1. Motlakas...,{'text': 'A. Tindzaba Letibalulekile Telive1....,{'text': 'A. Mafhungo a Zwino kha Shango1. F...,{'text': 'A. Xiyimo xa Khabinete eka Timhakaku...
3,Statement on the virtual Cabinet Meeting of 7 ...,https://www.gov.za/speeches/statement-virtual-...,2022-07-11,2022-07-11 00:00:00+02:00,{'text': 'A. Issues in the Environment1. Updat...,{'text': 'A. Kwessies in die omgewing1. ...,{'text': 'A. Ezingundabamlonyeni 1. ...,{'text': 'A. Imiba engundaba-mlonyeni1. ...,{'text': 'A. Izindaba ezisematheni1. Eza...,{'text': 'A. Maemo a Renang ha Jwale1. T...,{'text': 'A. Merero ye Bohlokwa Seemong sa Bja...,{'text': 'A. Merero ya ga Jaanong1. Tshe...,{'text': 'A. Tindzaba Talelive1. Takamuv...,{'text': 'A. Mafhungo a Zwino kha Shango1. Mas...,{'text': 'A. Xiyimo xa Khabinete eka timhak...
4,Statement on the Cabinet Meeting of 22 June 2022,https://www.gov.za/speeches/statement-virtual-...,2022-06-23,2022-06-23 00:00:00+02:00,{'text': 'A. ISSUES IN THE ENVIRONMENT1. ...,{'text': 'A. Kwessies in die Omgewing1. ...,{'text': 'A. Ezingundabamlonyeni 1. Ukutjhugu...,{'text': 'A. Imiba enguNdaba-mlonyeni1. ...,{'text': 'A. Ezisematheni1. Ukuguquguquk...,{'text': 'A. Ditaba tse Itlhommeng Pele 1. Dip...,{'text': 'A. Merero ye Bohlokwa Seemong sa ...,{'text': 'A. Merero ya ga Jaanong1. Phetogo y...,{'text': 'A. Tindzaba Letibalulekile TaleLive1...,{'text': 'A. Mafhungo a Zwino kha Shango1. Tsh...,{'text': 'A. Xiyimo xa Khabinete eka timhak...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Statement on the Cabinet meeting of 26 June 2013,https://www.gov.za/statement-cabinet-meeting-2...,2013-07-02,2013-07-02 00:00:00+02:00,{'text': '1. Cabinet’s position on prominent i...,{'text': '1. Die Kabinet se standpunt oor bela...,{'text': '1. Ubujamo beKhabinethi ngeendaba ez...,{'text': '1. Uluvo lweKhabhinethi kwimiba ebal...,{'text': '1. Isimo seKhabhinethi ezindabeni ez...,{'text': '1. Maemo a Kabinete mabapi le dintlh...,{'text': '1. Maemo a Kabinete ka ga merero ye ...,{'text': '1. Seemo sa Kabinete mo mererong e e...,{'text': '1. Tincumo teKhabhinethi mayel

In [11]:
from google.colab import files
files.download('/content/govza-cabinet-statements.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>